## **Config (paths, model names, imports)**


In [15]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Suppress HuggingFace tokenizers parallelism warning in Jupyter notebooks
# This prevents the "forked process" warning when using HuggingFaceEmbeddings
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Set LangChain configuration from environment variables
os.environ['LANGSMITH_TRACING'] = os.getenv('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = os.getenv('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')    
os.environ['LANGSMITH_PROJECT'] = os.getenv('LANGSMITH_PROJECT')

#API KEY
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [5]:
# Necessary packages
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import yaml
from pathlib import Path
from typing import Dict, List, Any

## **Extract metadata from mkdocs.yml**


In [6]:
# Load mkdocs.yml configuration
# Get the project root directory by checking for fastapi directory
current_dir = Path.cwd()
# Check if fastapi directory exists in current dir or parent
if (current_dir / "fastapi").exists():
    project_root = current_dir
elif (current_dir.parent / "fastapi").exists():
    project_root = current_dir.parent
else:
    # Fallback: assume parent directory (notebook is in rag-learning/rag-learning/)
    project_root = current_dir.parent

mkdocs_path = project_root / "fastapi" / "docs" / "en" / "mkdocs.yml"
docs_base_path = project_root / "fastapi" / "docs" / "en" / "docs"

# Convert to string for compatibility
mkdocs_path = str(mkdocs_path)
docs_base_path = str(docs_base_path)

# Pre-process the YAML file to handle the problematic !!python/name: tag
# We'll replace it with a null value since we only need the 'nav' section
import re

with open(mkdocs_path, 'r') as f:
    yaml_content = f.read()
    
# Replace the problematic Python tag with null using regex
# This handles the tag whether it's on its own or part of a value
yaml_content = re.sub(r'!!python/name:[^\s]+', 'null', yaml_content)

# Now parse the cleaned YAML
mkdocs_config = yaml.safe_load(yaml_content)

# Extract navigation structure
nav_structure = mkdocs_config.get('nav', [])

# Build a mapping from file paths to their metadata
def extract_nav_metadata(nav_items: List[Any], parent_path: List[str] = None) -> Dict[str, Dict[str, Any]]:
    """
    Recursively extract navigation metadata from mkdocs nav structure.
    Returns a dictionary mapping file paths to their metadata.
    """
    if parent_path is None:
        parent_path = []
    
    metadata_map = {}
    
    for item in nav_items:
        if isinstance(item, dict):
            # Handle dictionary items (e.g., {"Tutorial - User Guide": [...]})
            for key, value in item.items():
                if isinstance(value, list):
                    # Recursive case: nested navigation
                    new_path = parent_path + [key]
                    metadata_map.update(extract_nav_metadata(value, new_path))
                elif isinstance(value, str):
                    # Leaf case: key is section name, value is file path
                    file_path = value
                    metadata_map[file_path] = {
                        'section': key,
                        'category_path': parent_path + [key],
                        'top_level_category': parent_path[0] if parent_path else key,
                        'subcategory': parent_path[-1] if len(parent_path) > 1 else None,
                    }
        elif isinstance(item, str):
            # Handle simple string items (e.g., "tutorial/first-steps.md")
            file_path = item
            metadata_map[file_path] = {
                'section': None,
                'category_path': parent_path.copy(),
                'top_level_category': parent_path[0] if parent_path else 'Root',
                'subcategory': parent_path[-1] if parent_path else None,
            }
    
    return metadata_map

# Create the metadata mapping
nav_metadata_map = extract_nav_metadata(nav_structure)

# Display some examples
print(f"Total documents in navigation: {len(nav_metadata_map)}")
print("\nExample metadata entries:")
for i, (path, meta) in enumerate(list(nav_metadata_map.items())[:5]):
    print(f"\n{i+1}. {path}")
    print(f"   Category Path: {' > '.join(meta['category_path'])}")
    print(f"   Top Level: {meta['top_level_category']}")
    if meta['subcategory']:
        print(f"   Subcategory: {meta['subcategory']}")


Total documents in navigation: 144

Example metadata entries:

1. index.md
   Category Path: FastAPI
   Top Level: FastAPI

2. features.md
   Category Path: 
   Top Level: Root

3. learn/index.md
   Category Path: Learn
   Top Level: Learn
   Subcategory: Learn

4. python-types.md
   Category Path: Learn
   Top Level: Learn
   Subcategory: Learn

5. async.md
   Category Path: Learn
   Top Level: Learn
   Subcategory: Learn


## **Build dense vs lightweight corpora**


In [7]:
# Helper function to enrich documents with mkdocs metadata
def enrich_document_metadata(doc, metadata_map: Dict[str, Dict[str, Any]], docs_base_path: str):
    """Add mkdocs navigation metadata to a document based on its file path."""
    # Get the relative path from the document's source
    source_path = doc.metadata.get('source', '')
    
    # Convert absolute path to relative path from docs base
    if source_path.startswith(docs_base_path):
        relative_path = source_path[len(docs_base_path):].lstrip('/')
        
        # Normalize paths for comparison (handle both with/without .md extension)
        relative_path_normalized = relative_path.replace('.md', '')
        
        # Try exact match first
        if relative_path in metadata_map:
            nav_meta = metadata_map[relative_path]
        else:
            # Try matching by normalized path (without .md)
            nav_meta = None
            for nav_path, meta in metadata_map.items():
                nav_path_normalized = nav_path.replace('.md', '')
                # Match if paths are the same when normalized
                if nav_path_normalized == relative_path_normalized:
                    nav_meta = meta
                    break
                # Also try if the relative path ends with the nav path
                if relative_path.endswith(nav_path) or nav_path in relative_path:
                    nav_meta = meta
                    break
        
        # Add metadata if found
        if nav_meta:
            doc.metadata['section'] = nav_meta.get('section')
            doc.metadata['category_path'] = ' > '.join(nav_meta.get('category_path', []))
            doc.metadata['top_level_category'] = nav_meta.get('top_level_category')
            doc.metadata['subcategory'] = nav_meta.get('subcategory')
    
    return doc

# Dense corpus: tutorials + advanced guides
dense_loader = DirectoryLoader(
    docs_base_path,
    glob="tutorial/**/*.md",
    loader_cls=TextLoader,
)
dense_docs = dense_loader.load()
for d in dense_docs:
    d.metadata["corpus"] = "dense_docs"
    # Enrich with mkdocs metadata
    enrich_document_metadata(d, nav_metadata_map, docs_base_path)

# Lightweight corpus: e.g. fastapi-best-practices repo
faq_path = project_root / "fastapi-best-practices"
faq_loader = DirectoryLoader(
    str(faq_path),
    glob="README.md",
    loader_cls=TextLoader,
)
faq_docs = faq_loader.load()
for d in faq_docs:
    d.metadata["corpus"] = "faq_docs"

# Display enriched metadata
print("Sample enriched document metadata:")
print(dense_docs[2].metadata)

Sample enriched document metadata:
{'source': '/Users/dimitar/Desktop/Software_Dev/rag-learning/fastapi/docs/en/docs/tutorial/query-params.md', 'corpus': 'dense_docs', 'section': None, 'category_path': 'Learn > Tutorial - User Guide', 'top_level_category': 'Learn', 'subcategory': 'Tutorial - User Guide'}


In [8]:
from typing import Optional
from datetime import date
from pydantic import BaseModel, Field

class FastAPISearch(BaseModel):
    """
    Structured query for searching over FastAPI documentation chunks.
    The LLM will fill this from a natural-language question.
    """

    # What to search for semantically
    text: str = Field(
        ...,
        description=(
            "Main semantic search query over the document content. "
            "Use natural language describing the user's problem or question."
        ),
    )

    # High-level doc classification
    top_level_category: Optional[str] = Field(
        None,
        description=(
            "Top-level documentation category, such as 'Learn', "
            "'Reference', or 'Tutorials'. "
            "Use when the user seems to want conceptual/how-to material "
            "vs pure API reference."
        ),
    )

    subcategory: Optional[str] = Field(
        None,
        description=(
            "More specific documentation subcategory, such as "
            "'Tutorial - User Guide', 'Advanced User Guide', etc. "
            "Use when the user implicitly asks for guides or tutorials."
        ),
    )

    corpus: Optional[str] = Field(
        None,
        description=(
            "Internal corpus label (for example 'dense_docs', 'api_reference', "
            "'examples'). Use when the user implicitly wants a particular type "
            "of docs (e.g., reference vs prose)."
        ),
    )

    # Optional filters you might add later if your metadata has them
    earliest_publish_date: Optional[date] = Field(
        None,
        description=(
            "Earliest publish date for documents, inclusive. "
            "Use only if the user explicitly cares about recent or old docs."
        ),
    )

    latest_publish_date: Optional[date] = Field(
        None,
        description=(
            "Latest publish date for documents, exclusive. "
            "Use only if the user explicitly limits the time range."
        ),
    )

    def pretty_print(self) -> None:
        # Access model_fields from the class, not the instance (Pydantic v2.11+)
        model_fields = self.__class__.model_fields
        for field_name in model_fields.keys():
            field_value = getattr(self, field_name, None)
            field_info = model_fields[field_name]
            # Get default value, handling Pydantic v2 FieldInfo structure
            default_value = getattr(field_info, 'default', None)
            # Skip if value is None or equals the default
            if field_value is not None and field_value != default_value:
                print(f"{field_name}: {field_value}")


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

system = """You are an assistant that converts natural language questions
into structured search queries for the FastAPI documentation.

Your job is to:
- Extract a semantic search text that captures the user's problem or topic.
- Set metadata fields (top_level_category, subcategory, corpus, etc.)
  only when they are clearly implied by the question.

If you are unsure about a field, leave it empty (null).
Do NOT invent metadata values that are not supported by the schema.
Do NOT try to expand or reinterpret unknown acronyms; keep them as-is."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)  # or any chat model
structured_llm = llm.with_structured_output(FastAPISearch)
query_analyzer = prompt | structured_llm


## **Chunk + embed + index (two vector stores)**


In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
    chunk_overlap=150,
    length_function=len,)

# dense_docs and faq_docs already loaded and metadata-enriched earlier
dense_chunks = text_splitter.split_documents(dense_docs)
faq_chunks = text_splitter.split_documents(faq_docs)

print(f"Dense chunks: {len(dense_chunks)}")
print(f"FAQ chunks:   {len(faq_chunks)}")


Dense chunks: 388
FAQ chunks:   42


In [11]:
# Embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [12]:
vectordb_dir = Path("chroma_fastapi")
vectordb_dir.mkdir(exist_ok=True)

# Create vector stores with automatic persistence
# When persist_directory is provided, Chroma automatically persists to disk
dense_vs = Chroma.from_documents(
    documents=dense_chunks,
    embedding=embeddings, 
    collection_name="fastapi_dense_docs",
    persist_directory=str(vectordb_dir / "dense"),
)

faq_vs = Chroma.from_documents(
    documents=faq_chunks,
    embedding=embeddings, 
    collection_name="fastapi_faq_docs",
    persist_directory=str(vectordb_dir / "faq"),
)


In [24]:
# Create retrievers with search configuration
# Using the modern LangChain API with search_kwargs
dense_retriever = dense_vs.as_retriever(search_kwargs={"k": 8})
faq_retriever = faq_vs.as_retriever(search_kwargs={"k": 6})


## **Implement routing (which corpus, which strategy)**


In [25]:
def route_query(query: str) -> Dict[str, str]:
    """
    Route a query to the appropriate corpus(es) using rule-based logic.
    
    NOTE: faq_docs corpus is limited (single README.md), so routing is conservative.
    Only routes to faq_docs for very specific error/troubleshooting queries.
    
    Args:
        query: The user's search query
        
    Returns:
        Dictionary with "corpus" key set to "dense_docs", "faq_docs", or "both"
    """
    query_lower = query.lower()
    
    # Step 1: Keyword-based routing
    # FAQ indicators: STRONG error/debugging signals only (faq_docs is limited)
    # Only route to faq_docs for very specific troubleshooting queries
    strong_faq_keywords = [
        "500", "404", "403", "401", "400",  # HTTP error codes
        "stack trace", "traceback",  # Specific error indicators
        "doesn't work", "not working", "broken",  # Clear problem statements
        "troubleshoot", "debug",  # Explicit debugging intent
    ]
    
    # Moderate FAQ indicators: may use "both" for these
    moderate_faq_keywords = [
        "error", "exception", "failed", "failure", "crash", "issue", "bug", "problem"
    ]
    
    # Dense docs indicators: tutorials, guides, concepts, best practices
    # Note: "best practices" queries go to dense_docs since faq_docs is limited
    dense_keywords = [
        "tutorial", "how to", "how do", "guide", "best practices", "best practice",
        "architecture", "design", "deployment", "security", "dependency injection",
        "background tasks", "middleware", "async", "asynchronous", "concurrency",
        "testing", "cors", "authentication", "authorization", "validation",
        "pydantic", "openapi", "swagger", "websocket", "websockets",
        "project structure", "structure", "conventions", "patterns"
    ]
    
    # Count matches
    strong_faq_score = sum(1 for keyword in strong_faq_keywords if keyword in query_lower)
    moderate_faq_score = sum(1 for keyword in moderate_faq_keywords if keyword in query_lower)
    dense_score = sum(1 for keyword in dense_keywords if keyword in query_lower)
    
    # Step 2: Query length analysis
    query_length = len(query.split())
    is_short_query = query_length <= 3
    is_long_query = query_length > 10
    
    # Step 3: Decision logic (conservative - favor dense_docs)
    
    # Very strong FAQ signal (error codes, explicit debugging) → faq_docs
    if strong_faq_score > 0:
        return {"corpus": "faq_docs"}
    
    # Strong dense signal → dense_docs
    if dense_score > 0 and dense_score > moderate_faq_score:
        return {"corpus": "dense_docs"}
    
    # Moderate FAQ + dense signals → use both (faq might have relevant troubleshooting)
    if moderate_faq_score > 0 and dense_score > 0:
        return {"corpus": "both"}
    
    # Moderate FAQ only, but query is long/specific → both (to be safe)
    if moderate_faq_score > 0 and is_long_query:
        return {"corpus": "both"}
    
    # Moderate FAQ only, short query → dense_docs (faq_docs too limited)
    if moderate_faq_score > 0:
        return {"corpus": "dense_docs"}
    
    # Short, generic queries → dense_docs (tutorials/guides)
    if is_short_query:
        return {"corpus": "dense_docs"}
    
    # Default: dense_docs (more comprehensive corpus)
    return {"corpus": "dense_docs"}


def get_retriever_for_query(query: str, dense_retriever, faq_retriever):
    """
    Get the appropriate retriever(s) based on query routing.
    
    Args:
        query: The user's search query
        dense_retriever: Retriever for dense_docs corpus
        faq_retriever: Retriever for faq_docs corpus
        
    Returns:
        Single retriever or list of retrievers based on routing decision
    """
    routing = route_query(query)
    corpus = routing["corpus"]
    
    if corpus == "dense_docs":
        return dense_retriever
    elif corpus == "faq_docs":
        return faq_retriever
    else:  # "both"
        return [dense_retriever, faq_retriever]


## **Implement query translation (multi-query, maybe HyDE later)**


In [26]:
import tiktoken
import re
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Initialize tokenizer for token counting
# Using cl100k_base encoding (used by GPT-4, GPT-3.5-turbo)
try:
    encoding = tiktoken.get_encoding("cl100k_base")
except:
    # Fallback to a simpler encoding if cl100k_base not available
    encoding = tiktoken.get_encoding("gpt2")

# Constants
SHORT_THRESHOLD = 15  # tokens

def is_concrete_query(query: str) -> bool:
    """
    Detect if a query is concrete (has specific details like error codes, paths, endpoints).
    
    Args:
        query: The user's search query
        
    Returns:
        True if query is concrete, False if conceptual
    """
    query_lower = query.lower()
    
    # Error codes
    error_codes = ["500", "404", "403", "401", "400", "502", "503", "504"]
    if any(code in query_lower for code in error_codes):
        return True
    
    # File paths and extensions
    path_indicators = ["/", "\\", ".py", ".md", ".json", ".yaml", ".yml", "file:", "path:"]
    if any(indicator in query_lower for indicator in path_indicators):
        return True
    
    # API endpoints
    endpoint_indicators = ["/api/", "/users/", "/posts/", "endpoint", "route", "get ", "post ", "put ", "delete "]
    if any(indicator in query_lower for indicator in endpoint_indicators):
        return True
    
    # Code snippets (backticks, function names with parentheses)
    if "`" in query or "(" in query and ")" in query:
        return True
    
    # Specific IDs or versions (numbers that look like IDs)
    # Pattern for UUIDs, version numbers, or IDs
    if re.search(r'\b\d{4,}\b', query) or re.search(r'[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}', query_lower):
        return True
    
    return False


def count_tokens(text: str) -> int:
    """
    Count tokens in text using tiktoken.
    
    Args:
        text: Input text
        
    Returns:
        Number of tokens
    """
    return len(encoding.encode(text))


def route_query_translation(query: str) -> str:
    """
    Route a query to the appropriate translation method.
    
    Returns:
        "hyde", "multi_query", or "base"
    """
    token_count = count_tokens(query)
    is_concrete = is_concrete_query(query)
    
    # Rule 1: Concrete queries (error codes, paths, endpoints) → multi_query
    if is_concrete:
        return "multi_query"
    
    # Rule 2: Short conceptual queries → HyDE (needs expansion)
    if token_count < SHORT_THRESHOLD:
        return "hyde"
    
    # Rule 3: Medium/long queries → multi_query (benefits from variations)
    return "multi_query"


# HyDE: Hypothetical Document Embeddings
# Generate a hypothetical answer first, then search with that
hyde_template = """You are an AI assistant helping users find information about FastAPI.

Given the user's question below, write a concise hypothetical answer that would be found in FastAPI documentation. 
This hypothetical answer should be informative and directly address the question.

User question: {question}

Hypothetical answer:"""

hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
hyde_chain = hyde_prompt | llm | StrOutputParser()


# Multi-query: Generate multiple query variations
multi_query_template = """You are an AI language model assistant. Your task is to generate three 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 

Provide these alternative questions separated by newlines. Do not number them or add prefixes.

Original question: {question}"""

multi_query_prompt = ChatPromptTemplate.from_template(multi_query_template)
multi_query_chain = multi_query_prompt | llm | StrOutputParser()


def translate_query(query: str, method: str = None) -> List[str]:
    """
    Translate a query using the specified method.
    
    Args:
        query: Original user query
        method: "hyde", "multi_query", or "base". If None, routes automatically.
        
    Returns:
        List of queries to use for retrieval
    """
    if method is None:
        method = route_query_translation(query)
    
    if method == "hyde":
        # Generate hypothetical answer
        hypothetical_answer = hyde_chain.invoke({"question": query})
        # Use the hypothetical answer as the search query
        return [hypothetical_answer]
    
    elif method == "multi_query":
        # Generate multiple query variations
        variations_text = multi_query_chain.invoke({"question": query})
        # Split by newlines and clean up
        variations = [v.strip() for v in variations_text.split("\n") if v.strip()]
        # Include original query + variations
        return [query] + variations[:3]  # Limit to 3 variations
    
    else:  # "base"
        # No translation, use original query
        return [query]

## **Retrieval + fusion + answer generation**


## **Small evaluation loop**
